<a href="https://colab.research.google.com/github/SERVIR/flood_mapping_intercomparison/blob/main/hydrafloods/training_materials/oct_2021_hf_training/notebooks/supplementary/water_fraction_downscaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# WARNING: this is a work in progress, please come back later to see workflow

### Water fraction downscaling

use the Dynamica Nearest Neighbor Search (DNNS) algorithm to calculate water fraction within a pixel and use terrain information to

In [ ]:
# get the Sentinel 1 collection as a hydrafloods Dataset
viirs = hf.Viirs(region,"2020-11-01","2020-12-01")

viirs.apply_func(hf.add_indices,indices=["mndwi"],inplace=True)

In [ ]:
water_proba = hf.multidim_semisupervised(viirs.collection.median(),bands=["red","nir","swir1","mndwi"],rank_band="mndwi",ranking="max",n_samples=1500,region=region)

In [ ]:
# view the results of SAR water mapping
Map = geemap.Map(center=(16.0029, -90.5109), zoom=12)

Map.addLayer(viirs.collection.median(), optical_vis, 'VIIRS')
Map.addLayer(viirs.collection.median().select("mndwi"), wi_vis, 'VIIRS mndwi')
Map.addLayer(water_proba,{"min":0,"max":1,"palette":cm.palettes.inferno}, "VIIRS (water proba)")


Map.addLayerControl()
Map

In [ ]:
def dnns(img):
    kernel_scale = 40
    pixel_scale = img.projection().nominalScale()

    k1 = ee.Kernel.square(kernel_scale, 'pixels', False)

    water_proba = hf.multidim_semisupervised(img,bands=["red","nir","swir1","mndwi"],rank_band="mndwi",ranking="max",n_samples=1500)

    water = water_proba.gt(0.9)
    land = water_proba.lt(0.1)

    mix = water.Not().And(land.Not())

    ave_water = img.updateMask(water)

    N_nmin_water = 1
    N_water = water.convolve(k1)

    water_rf = ave_water.convolve(k1).multiply(N_water.gte(N_nmin_water)).divide(N_water)
    water_rf = water_rf.add(ave_water_img.multiply(water_rf.Not()))

    ave_land = img.updateMask(land)

    R1 = img['I1'].divide(img['I3'])
    R2 = img['I2'].divide(img['I3'])
    R3 = R1.subtract(water_rf.select('I1').divide(img['I3']) )
    R4 = R2.subtract(water_rf.select('I2').divide(img['I3']) )
